<a href="https://colab.research.google.com/github/svhenrique/house-prices-predictor/blob/main/HousePricesPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando Bibliotécas necessárias 

In [35]:
# montando drive (para o colab)
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [36]:
# importando bibliotecas necessárias
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## Lendo dados

In [50]:
# dados em pasta 
pasta = '/content/drive/My Drive/datasets/house-prices/'

In [51]:
# lendo dados de treino e teste
train_data = pd.read_csv(pasta + 'train.csv')
test_data = pd.read_csv(pasta + 'test.csv')

## Limpeza e organização dos dados

In [52]:
# mostrando shape resultante
print('Train data shape: ', train_data.shape)
print('Test data shape: ', test_data.shape)

Train data shape:  (1460, 81)
Test data shape:  (1459, 80)


In [53]:
# removendo dados nan de train_data
train_data.dropna(how='any',axis=0)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [54]:
# removendo dados nan de test_data
test_data.dropna(how='any',axis=0)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


É possível observar que todas as linhas foram apagadas, isso indica que existem valores "nan" em todas as instâncias. Entretanto, isso é uma confusão que o pandas fez ao ler o atributo "NA" de "Alley", que significa "No alley access".

In [55]:
# observando as features com maiores ocorrências de nan 
train_data.isnull().sum()[train_data.isnull().sum() > len(train_data)/2]

Alley          1369
PoolQC         1453
Fence          1179
MiscFeature    1406
dtype: int64

Logo, essas 4 features podem ter valores normais que são lidos como nan.

Eu irei seguir o glossário de valores para identificar atributos como "NA" que realmente significam algo, evitando perca de dados.

In [56]:
# Existem valores NA, que são confundidos com nans, sobre nas colunas a seguir
print(train_data['Alley'].unique())
print(train_data['PoolQC'].unique())
print(train_data['Fence'].unique())
print(train_data['MiscFeature'].unique())
print(train_data['BsmtQual'].unique())
print(train_data['BsmtCond'].unique())
print(train_data['BsmtExposure'].unique())
print(train_data['BsmtFinType1'].unique())
print(train_data['BsmtFinType2'].unique())
print(train_data['FireplaceQu'].unique())
print(train_data['GarageType'].unique())
print(train_data['GarageFinish'].unique())
print(train_data['GarageQual'].unique())
print(train_data['GarageCond'].unique())

[nan 'Grvl' 'Pave']
[nan 'Ex' 'Fa' 'Gd']
[nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
[nan 'Shed' 'Gar2' 'Othr' 'TenC']
['Gd' 'TA' 'Ex' nan 'Fa']
['TA' 'Gd' nan 'Fa' 'Po']
['No' 'Gd' 'Mn' 'Av' nan]
['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
[nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
['RFn' 'Unf' 'Fin' nan]
['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
['TA' 'Fa' nan 'Gd' 'Po' 'Ex']


In [57]:
# inserindo o valor "NA" (No alley access) para os nan em Alley de treino e teste
train_data['Alley'] = train_data['Alley'].fillna('NA')
test_data['Alley'] = test_data['Alley'].fillna('NA')

In [58]:
# inserindo o valor "NA" (No Pool) para os nan em Alley de treino e teste
train_data['PoolQC'] = train_data['PoolQC'].fillna('NA')
test_data['PoolQC'] = test_data['PoolQC'].fillna('NA')

In [59]:
# inserindo o valor "NA" (No Fencel) para os nan em PoolQC de treino e teste
train_data['Fence'] = train_data['Fence'].fillna('NA')
test_data['Fence'] = test_data['Fence'].fillna('NA')

In [60]:
# inserindo o valor "NA" (None) para os nan em PoolQC de treino e teste
train_data['MiscFeature'] = train_data['MiscFeature'].fillna('NA')
test_data['MiscFeature'] = test_data['MiscFeature'].fillna('NA')

In [61]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtQual de treino e teste
train_data['BsmtQual'] = train_data['BsmtQual'].fillna('NA')
test_data['BsmtQual'] = test_data['BsmtQual'].fillna('NA')

In [62]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtCond de treino e teste
train_data['BsmtCond'] = train_data['BsmtCond'].fillna('NA')
test_data['BsmtCond'] = test_data['BsmtCond'].fillna('NA')

In [63]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtExposure de treino e teste
train_data['BsmtExposure'] = train_data['BsmtExposure'].fillna('NA')
test_data['BsmtExposure'] = test_data['BsmtExposure'].fillna('NA')

In [64]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtFinType1 de treino e teste
train_data['BsmtFinType1'] = train_data['BsmtFinType1'].fillna('NA')
test_data['BsmtFinType1'] = test_data['BsmtFinType1'].fillna('NA')

In [65]:
# inserindo o valor "NA" (No Basement) para os nan em BsmtFinType2 de treino e teste
train_data['BsmtFinType2'] = train_data['BsmtFinType2'].fillna('NA')
test_data['BsmtFinType2'] = test_data['BsmtFinType2'].fillna('NA')

In [66]:
# inserindo o valor "NA" (No Fireplace) para os nan em FireplaceQu de treino e teste
train_data['FireplaceQu'] = train_data['FireplaceQu'].fillna('NA')
test_data['FireplaceQu'] = test_data['FireplaceQu'].fillna('NA')

In [67]:
# inserindo o valor "NA" (No Garage) para os nan em GarageType de treino e teste
train_data['GarageType'] = train_data['GarageType'].fillna('NA')
test_data['GarageType'] = test_data['GarageType'].fillna('NA')

In [68]:
# inserindo o valor "NA" (No Garage) para os nan em GarageFinish de treino e teste
train_data['GarageFinish'] = train_data['GarageFinish'].fillna('NA')
test_data['GarageFinish'] = test_data['GarageFinish'].fillna('NA')

In [69]:
# inserindo o valor "NA" (No Garage) para os nan em GarageQual de treino e teste
train_data['GarageQual'] = train_data['GarageQual'].fillna('NA')
test_data['GarageQual'] = test_data['GarageQual'].fillna('NA')

In [70]:
# inserindo o valor "NA" (No Garage) para os nan em GarageCond de treino e teste
train_data['GarageCond'] = train_data['GarageCond'].fillna('NA')
test_data['GarageCond'] = test_data['GarageCond'].fillna('NA')

In [71]:
# removendo os dados nan verdadeiros dos datasets
train_data = train_data.dropna(how='any',axis=0)
test_data = test_data.dropna(how='any',axis=0)

In [72]:
# mostrando shape resultante
print('Train data shape: ', train_data.shape)
print('Test data shape: ', test_data.shape)

Train data shape:  (1120, 81)
Test data shape:  (1139, 80)


In [75]:
# removendo coluna id 
train_data.drop('Id', inplace=True, axis=1)
test_data.drop('Id', inplace=True, axis=1)